# Rename and Merge

Now that we have two aggregated dataframes for Comments and Answers that map to a Question, our final statistical challenge is to merge these two dataframes, combine the scores and find the questions with the highest aggregate scores overall. Note that there are two key problems to overcome, firstly one dataframe uses the column header 'ParentId' while the other uses 'PostId'. So we should rename 'ParentId' to 'PostId' in the Answers dataframe. The second problem is that both dataframes have the column 'Score'. But we can't have two columns in a single dataframe with the same key. So we should rename them to 'Answers-Score' and 'Comments-Score'. Once this is done, we can merge the two datafraes by PostId and create a new column called 'Total-Score' which contains the sum of the other two Score Columns. Once we have everything in place, we can sort based on Total-Score, and write out our new dataframe, along with a dump of the PostIds and Total Scores to a text file for a sanity check.

### Read a feather file and return a dataframe. This is already done for you. You just have to call it from main to convert a feather file into a dataframe.

In [1]:
import pyarrow.feather as feather
import pandas as pd

In [2]:
def arrow_to_df(input_file_name):
    df = feather.read_feather(input_file_name)
    return df

### Write a feather file using a dataframe. This is done for you, you just need to call it.

In [3]:
def df_to_arrow(output_file_name, df):
    feather.write_feather(df, output_file_name, compression='zstd')
    return

### This function will write out only the PostId and Total-Score columns from a dataframe. This assumes of course that you have named the columns correctly!

In [4]:
def write_total_scores(df,out_file):
    with open(out_file,'w') as f:
        fdf = df[['PostId','Total-Score']].copy()
        f.write(fdf.to_string(index=False))
        return
        

### The sort function from Question 2.

In [5]:
def sort_df (df,key):
    new_df = df.sort_values(by=[key],ascending=False ,ignore_index=True)
    
    return new_df

### Main Loop:
* First read the Answers-Sum.feather and Comments-Sum.feather files into dataframes using the provided function.
* Now rename the columns as defined in the header above.
* Once the header names are correct, you can merge based on 'PostId'.
* Then, you can create a new column called 'Total-Score' that is the sum of the other two Score columns.
* Sort the new dataframe using the sort function from Q2 based on 'Total-Score'.
* Finally, you need to write_total_scores() for the new dataframe. The output file name should be "Posts-Total-Scores.txt". This will by our sanity check that you got the split function correct. Make sure you use the output file names exactly as shown (case sensitive).
* We do not need to write out the final merged dataframe as a feather file as we won't need it for the final challenge.

In [6]:

def main():
    
    #Read Ans-sum and Comments-sum to dataframes
    sum_ans = arrow_to_df('Answers-Sum.feather')
    sum_comms = arrow_to_df('Comments-Sum.feather')
    
    #Re-naming columns
    sum_ans =sum_ans.rename(columns={'Score':'Answers-Score'})
    sum_ans =sum_ans.rename(columns={'ParentId':'PostId'})
    sum_comms = sum_comms.rename(columns={'Score':'Comments-Score'})
    
    #Merging based on PostId
    merge_df =pd.merge(sum_ans,sum_comms,on='PostId')
    
    #Sum of comment-score + Answer-score
    merge_df['Total-Score'] =  merge_df['Comments-Score']+ merge_df['Answers-Score']
    
    #Soring done based on Total-Score
    sort_df(merge_df,'Total-Score')
    
    #output final file
    write_total_scores(merge_df,'Posts-Total-Scores.txt')
    return

In [7]:

if __name__ == '__main__':
    main()
    print ('[INFO]:Script completed with no errors')

[INFO]:Script completed with no errors
